In [ ]:
# Libs
import os
import pandas as pd
from download_transformacao_CNPJ import EXTRATOR_CNPJ
from pyspark.sql.functions import concat_ws, lpad, coalesce, when, lit
from time import localtime, strftime
current_dir = os.getcwd()
dir_dados = os.path.join(current_dir, r"dados")

In [ ]:
# Se passar baixar_e_extrair como false, precisa do nome do arquivo.
print(f'Começando a buscar os dados: {strftime("%d/%m/%Y %H:%M:%S", localtime())}')
ESTABELECIMENTOS, spark = EXTRATOR_CNPJ(baixar_e_extrair=False, nome_arquivo="Estabelecimentos").run()
print(f'Termino da coleta dos ESTABELECIMENTOS: {strftime("%d/%m/%Y %H:%M:%S", localtime())}')
EMPRESAS, spark = EXTRATOR_CNPJ(baixar_e_extrair=False, nome_arquivo="Empresas").run()
print(f'Termino da coleta dos EMPRESAS: {strftime("%d/%m/%Y %H:%M:%S", localtime())}')
MUNICIPIOS, spark = EXTRATOR_CNPJ(baixar_e_extrair=False, nome_arquivo="Municipios").run()
print(f'Termino da coleta dos MUNICIPIOS: {strftime("%d/%m/%Y %H:%M:%S", localtime())}')
SOCIOS, spark = EXTRATOR_CNPJ(baixar_e_extrair=False, nome_arquivo="Socios").run()
print(f'Termino da coleta dos SOCIOS: {strftime("%d/%m/%Y %H:%M:%S", localtime())}')
SIMPLES, spark = EXTRATOR_CNPJ(baixar_e_extrair=False, nome_arquivo="Simples").run()
print(f'Final da coleta dos dados: {strftime("%d/%m/%Y %H:%M:%S", localtime())}')

In [ ]:
CNAES = {
        5620104:'Fornecimento de alimentos preparados preponderantemente para consumo domiciliar',
        5611201:'Restaurantes e similares',
        5611203:'Lanchonetes casas de chá de sucos e similares',
        5611204:'Bares e outros estabelecimentos especializados em servir bebidas sem entretenimento',
        5611205:'Bares e outros estabelecimentos especializados em servir bebidas com entretenimento',
        4721102:'Padaria e confeitaria com predominância de revenda'
        }

In [ ]:
cnaes = pd.DataFrame(CNAES.items(),columns=["CNAE_PRINCIPAL", "CNAE_DESCRICAO"], dtype="string")
cnaes

In [ ]:
def filtragem_cnae_sql(cod_cnae: int):
    # Cria uma view com o mesmo nome do DataFrame
    ESTABELECIMENTOS.createOrReplaceTempView("ESTABELECIMENTOS")
    EMPRESAS.createOrReplaceTempView("EMPRESAS")
    MUNICIPIOS.createOrReplaceTempView("MUNICIPIOS")
    SIMPLES.createOrReplaceTempView("SIMPLES")
    SOCIOS.createOrReplaceTempView("SOCIOS")
    # cria um dataframe com base na query
    dataframe = spark.sql(
        f"""
        SELECT CONCAT(LPAD(EST.CNPJ_BASE, 8, '0'), LPAD(EST.CNPJ_ORDEM, 4, '0'), LPAD(EST.CNPJ_DV, 2, '0')) AS CNPJ,
                EMP.RAZAO_SOCIAL,
                EST.NOME_FANTASIA,
                EST.SITUACAO_CADASTRAL,
                EST.DATA_SITUACAO_CADASTRAL,
                EST.DATA_INICIO_ATIVIDADE,
                SOC.NOME_SOCIO,
                SIM.OPCAO_MEI,
                SIM.DATA_OPCAO_MEI,
                SIM.DATA_EXCLUSAO_MEI,
                EST.CNAE_PRINCIPAL,
                CONCAT(
                    COALESCE(EST.TIPO_LOGRADOURO, ''),
                    ' ',
                    COALESCE(EST.LOGRADOURO, ''),
                    ' ',
                    COALESCE(EST.NUMERO, ''),
                    ' ',
                    COALESCE(EST.COMPLEMENTO, '')
                ) AS ENDERECO
                EST.BAIRRO,
                MUN.NOME_MUNICIPIO AS CIDADE,
                EST.UF,
                EST.CEP,
                CONCAT(
                    COALESCE(EST.DDD_CONTATO, ''), 
                    ' ',
                    COALESCE(EST.TELEFONE_CONTATO, '')
                    ) AS TELEFONE,
                EST.EMAIL
        FROM ESTABELECIMENTOS AS EST, EMPRESAS AS EMP, MUNICIPIOS AS MUN, SIMPLES AS SIM, SOCIOS AS SOC
        WHERE EST.CNPJ_BASE = EMP.CNPJ_BASE
            AND EST.CNPJ_BASE = SIM.CNPJ_BASE
            AND EST.CNPJ_BASE = SOC.CNPJ_BASE
            AND EST.CODIGO_MUNICIPIO = MUN.CODIGO_MUNICIPIO
            AND EST.CNAE_PRINCIPAL = {cod_cnae}
            AND EST.SITUACAO_CADASTRAL IN (2, 3, 4)
            AND SIM.OPCAO_MEI IS NOT "N"
        """
    )
    #display(dataframe.show(5))
    return dataframe

In [ ]:
def filtragem_cnae_df(cod_cnae:int):
    # cria um dataframe com base nos filtros aplicados
    from pyspark.sql.functions import concat_ws, lpad, coalesce, when
    dataframe = (
        ESTABELECIMENTOS
        .join(EMPRESAS, "CNPJ_BASE", "left")
        .join(MUNICIPIOS, "CODIGO_MUNICIPIO", "left")
        .join(SIMPLES, "CNPJ_BASE", "left")
        .join(SOCIOS, "CNPJ_BASE", "left")
        .where(
            (ESTABELECIMENTOS["CNAE_PRINCIPAL"] == f"{cod_cnae}") &
            (ESTABELECIMENTOS["SITUACAO_CADASTRAL"].isin([2, 3, 4])) &
            (SIMPLES["OPCAO_MEI"] == "S")|
            (ESTABELECIMENTOS["CNAE_SECUNDARIO"].isin(cod_cnae)) &
            (ESTABELECIMENTOS["SITUACAO_CADASTRAL"].isin([2, 3, 4])) &
            (SIMPLES["OPCAO_MEI"] == "S")
        )
        .select(
            concat_ws("", 
                lpad(ESTABELECIMENTOS["CNPJ_BASE"].cast("bigint"), 8, "0"), 
                lpad(ESTABELECIMENTOS["CNPJ_ORDEM"], 4, "0"), 
                lpad(ESTABELECIMENTOS["CNPJ_DV"], 2, "0")
            ).alias("CNPJ"),
            when(EMPRESAS.RAZAO_SOCIAL.isNull(), None).otherwise(EMPRESAS.RAZAO_SOCIAL).alias("RAZAO_SOCIAL"),
            when(ESTABELECIMENTOS.NOME_FANTASIA.isNull(), None).otherwise(ESTABELECIMENTOS.NOME_FANTASIA).alias("NOME_FANTASIA"),
            ESTABELECIMENTOS.SITUACAO_CADASTRAL,
            ESTABELECIMENTOS.DATA_SITUACAO_CADASTRAL,
            ESTABELECIMENTOS.DATA_INICIO_ATIVIDADE,
            when(SOCIOS.NOME_SOCIO.isNull(), None).otherwise(SOCIOS.NOME_SOCIO).alias("NOME_SOCIO"),
            when(SIMPLES.OPCAO_MEI.isNull(), None).otherwise(SIMPLES.OPCAO_MEI).alias("OPCAO_MEI"),
            when(SIMPLES.DATA_EXCLUSAO_MEI.isNull(), None).otherwise(SIMPLES.DATA_EXCLUSAO_MEI).alias("DATA_EXCLUSAO_MEI"),
            when(SIMPLES.DATA_OPCAO_MEI.isNull(), None).otherwise(SIMPLES.DATA_OPCAO_MEI).alias("DATA_OPCAO_MEI"),
            ESTABELECIMENTOS.CNAE_PRINCIPAL,
            ESTABELECIMENTOS.CNAE_SECUNDARIO,
            concat_ws(" ",
                when(ESTABELECIMENTOS.TIPO_LOGRADOURO.isNull(), None).otherwise(ESTABELECIMENTOS.TIPO_LOGRADOURO),
                when(ESTABELECIMENTOS.LOGRADOURO.isNull(), None).otherwise(ESTABELECIMENTOS.LOGRADOURO),
                when(ESTABELECIMENTOS.NUMERO.isNull(), None).otherwise(ESTABELECIMENTOS.NUMERO),
                when(ESTABELECIMENTOS.COMPLEMENTO.isNull(), None).otherwise(ESTABELECIMENTOS.COMPLEMENTO)
            ).alias("ENDERECO"),
            ESTABELECIMENTOS.BAIRRO,
            MUNICIPIOS.NOME_MUNICIPIO.alias("CIDADE"),
            ESTABELECIMENTOS.UF,
            ESTABELECIMENTOS.CEP,
            concat_ws("-", 
                when(ESTABELECIMENTOS.DDD_CONTATO.isNull(), None).otherwise(ESTABELECIMENTOS.DDD_CONTATO),
                when(ESTABELECIMENTOS.TELEFONE_CONTATO.isNull(), None).otherwise(ESTABELECIMENTOS.TELEFONE_CONTATO)
            ).alias("TELEFONE"),
            ESTABELECIMENTOS.EMAIL
        )
    )

    #display(dataframe.show(5))
    return dataframe

In [ ]:
def filtra_cnae_lista(lista_cnaes:list):
    # cria um dataframe com base nos filtros aplicados
    from pyspark.sql.functions import concat_ws, lpad, coalesce, when 
    dataframe = (
        ESTABELECIMENTOS
        .join(EMPRESAS, "CNPJ_BASE", "left")
        .join(MUNICIPIOS, "CODIGO_MUNICIPIO", "left")
        .join(SIMPLES, "CNPJ_BASE", "left")
        .join(SOCIOS, "CNPJ_BASE", "left")
        .where(
            (ESTABELECIMENTOS["CNAE_PRINCIPAL"].isin(lista_cnaes)) &
            (ESTABELECIMENTOS["SITUACAO_CADASTRAL"].isin([2, 3, 4])) &
            (SIMPLES["OPCAO_MEI"] == "S")|
            (ESTABELECIMENTOS["CNAE_SECUNDARIO"].isin(lista_cnaes)) &
            (ESTABELECIMENTOS["SITUACAO_CADASTRAL"].isin([2, 3, 4])) &
            (SIMPLES["OPCAO_MEI"] == "S")
        )
        .select(
            concat_ws("", 
                lpad(ESTABELECIMENTOS["CNPJ_BASE"].cast("bigint"), 8, "0"), 
                lpad(ESTABELECIMENTOS["CNPJ_ORDEM"], 4, "0"), 
                lpad(ESTABELECIMENTOS["CNPJ_DV"], 2, "0")
            ).alias("CNPJ"),
            when(EMPRESAS.RAZAO_SOCIAL.isNull(), None).otherwise(EMPRESAS.RAZAO_SOCIAL).alias("RAZAO_SOCIAL"),
            when(ESTABELECIMENTOS.NOME_FANTASIA.isNull(), None).otherwise(ESTABELECIMENTOS.NOME_FANTASIA).alias("NOME_FANTASIA"),
            ESTABELECIMENTOS.SITUACAO_CADASTRAL,
            ESTABELECIMENTOS.DATA_SITUACAO_CADASTRAL,
            ESTABELECIMENTOS.DATA_INICIO_ATIVIDADE,
            when(SOCIOS.NOME_SOCIO.isNull(), None).otherwise(SOCIOS.NOME_SOCIO).alias("NOME_SOCIO"),
            when(SIMPLES.OPCAO_MEI.isNull(), None).otherwise(SIMPLES.OPCAO_MEI).alias("OPCAO_MEI"),
            when(SIMPLES.DATA_EXCLUSAO_MEI.isNull(), None).otherwise(SIMPLES.DATA_EXCLUSAO_MEI).alias("DATA_EXCLUSAO_MEI"),
            when(SIMPLES.DATA_OPCAO_MEI.isNull(), None).otherwise(SIMPLES.DATA_OPCAO_MEI).alias("DATA_OPCAO_MEI"),
            ESTABELECIMENTOS.CNAE_PRINCIPAL,
            ESTABELECIMENTOS.CNAE_SECUNDARIO,
            concat_ws(" ",
                when(ESTABELECIMENTOS.TIPO_LOGRADOURO.isNull(), None).otherwise(ESTABELECIMENTOS.TIPO_LOGRADOURO),
                when(ESTABELECIMENTOS.LOGRADOURO.isNull(), None).otherwise(ESTABELECIMENTOS.LOGRADOURO),
                when(ESTABELECIMENTOS.NUMERO.isNull(), None).otherwise(ESTABELECIMENTOS.NUMERO),
                when(ESTABELECIMENTOS.COMPLEMENTO.isNull(), None).otherwise(ESTABELECIMENTOS.COMPLEMENTO)
            ).alias("ENDERECO"),
            ESTABELECIMENTOS.BAIRRO,
            MUNICIPIOS.NOME_MUNICIPIO.alias("CIDADE"),
            ESTABELECIMENTOS.UF,
            ESTABELECIMENTOS.CEP,
            concat_ws("-", 
                when(ESTABELECIMENTOS.DDD_CONTATO.isNull(), None).otherwise(ESTABELECIMENTOS.DDD_CONTATO),
                when(ESTABELECIMENTOS.TELEFONE_CONTATO.isNull(), None).otherwise(ESTABELECIMENTOS.TELEFONE_CONTATO)
            ).alias("TELEFONE"),
            ESTABELECIMENTOS.EMAIL
        )
    )

    #display(dataframe.show(5))
    return dataframe

In [ ]:
def salvar_df_cnae(CNAES:dict[int,str] = CNAES, opcao:int = None):
    from backup_limpeza import backup_limpeza_simples
    """
    Args:
        CNAES (dict[int,str], optional): informa um dicionário com os códigos e descrição cnae.
    Return:
        dados_pandas : salva o dataframe gerador pela função em um arquivo único csv e arquivos parquets
    """
    arquivo_csv = os.path.join(dir_dados, r"csv\BASE_RFB_MEI.csv")
    if os.path.exists(arquivo_csv):
        nome_backup = dir_dados + r"/backup/"
        if not os.path.exists(nome_backup):
            os.makedirs(nome_backup)
        backup_limpeza_simples(pasta=arquivo_csv.replace(r"BASE_RFB_MEI.csv", ""), nome_zipado=nome_backup + f"BASE_RFB_{strftime('%d-%m-%Y %H_%M_%S', localtime())}.zip")
    if opcao == 1:
        dados = None
        for cod_cnae, descricao_cnae in CNAES.items():
            if dados is None:
                print(f"{cod_cnae}:{descricao_cnae}")
                dados = filtragem_cnae_df(cod_cnae)
                dados = dados.withColumn(
                                "CNAE_DESCRICAO", lit(descricao_cnae.upper())
                            )
            else:
                print(f"{cod_cnae}:{descricao_cnae}")
                dados_incrementados = filtragem_cnae_df(cod_cnae)
                dados_incrementados = dados_incrementados.withColumn(
                    "CNAE_DESCRICAO", lit(descricao_cnae.upper())
                )
                dados = dados.union(dados_incrementados)
                dados = dados.sort("CNPJ")
        dados_pandas = dados.toPandas()
        print(len(dados_pandas.index))
        dados_pandas.to_csv(arquivo_csv, sep=";",mode="a", encoding="utf-8", index=False)
    elif opcao == 2:
        lista_cnaes = list(CNAES.keys())
        dados = filtra_cnae_lista(lista_cnaes)
        cnaes = pd.DataFrame(CNAES.items(),columns=["CNAE_PRINCIPAL", "CNAE_DESCRICAO"], dtype="string")
        dados_pandas = dados.toPandas()
        dados_pandas = pd.merge(dados_pandas, cnaes, on="CNAE_PRINCIPAL")
        
        dados_pandas = dados_pandas[["CNPJ","RAZAO_SOCIAL","NOME_FANTASIA",
            "SITUACAO_CADASTRAL","DATA_SITUACAO_CADASTRAL",
            "DATA_INICIO_ATIVIDADE","NOME_SOCIO","OPCAO_MEI","DATA_EXCLUSAO_MEI","DATA_OPCAO_MEI","CNAE_PRINCIPAL","ENDERECO",
            "BAIRRO","CIDADE","UF","CEP","TELEFONE","CNAE_DESCRICAO", "EMAIL"]]
        print(len(dados_pandas.index))
        dados_pandas.to_csv(arquivo_csv, sep=";",mode="a", encoding="utf-8", index=False)
    else:
        pass

In [ ]:
salvar_df_cnae(CNAES, opcao=2)

In [3]:
import pandas as pd
import os

current_dir = os.getcwd()
dir_dados = os.path.join(current_dir, r"dados")
MEIS = pd.read_csv(f"{current_dir}"+ r"\BASE_MEI_RFB_SEM_EMAIL_CONTABIL.csv", sep=";", low_memory=False, dtype="string")
MEIS["DATA_EXCLUSAO_MEI"].fillna("0", inplace=True)

MEIS[MEIS["DATA_EXCLUSAO_MEI"] == 0].count()

CNPJ                       0
RAZAO_SOCIAL               0
NOME_FANTASIA              0
SITUACAO_CADASTRAL         0
DATA_SITUACAO_CADASTRAL    0
DATA_INICIO_ATIVIDADE      0
NOME_SOCIO                 0
OPCAO_MEI                  0
DATA_EXCLUSAO_MEI          0
DATA_OPCAO_MEI             0
CNAE_PRINCIPAL             0
ENDERECO                   0
BAIRRO                     0
CIDADE                     0
UF                         0
CEP                        0
TELEFONE                   0
CNAE_DESCRICAO             0
EMAIL                      0
dtype: int64

In [11]:
lagoa_santa = MEIS.query("CIDADE == 'LAGOA SANTA' and UF == 'MG'")

lagoa_santa.to_excel("excel.xlsx")

In [17]:
def formata_cnpj(cnpj):
    cnpj = str(cnpj)
    cnpj = cnpj[:2] + '.' + cnpj[2:5] + '.' + cnpj[5:8] + '/' + cnpj[8:12] + '-' + cnpj[12:]
    return cnpj


In [20]:
MEIS["CNPJ"] = MEIS["CNPJ"].apply(formata_cnpj)
MEIS.head(2)

,CNPJ,RAZAO_SOCIAL,NOME_FANTASIA,SITUACAO_CADASTRAL,DATA_SITUACAO_CADASTRAL,DATA_INICIO_ATIVIDADE,NOME_SOCIO,OPCAO_MEI,DATA_EXCLUSAO_MEI,DATA_OPCAO_MEI,CNAE_PRINCIPAL,ENDERECO,BAIRRO,CIDADE,UF,CEP,TELEFONE,CNAE_DESCRICAO,EMAIL
0,00.251.353/0001-64,SONIA DA SILVA RUIZ BELTRAME,TFC REFEICOES,2,20040123,19941017,<NA>,S,0,20200101,5611201,RUA SUICA 901,PQ DAS NACOES,SANTO ANDRE,SP,09210000,<NA>,Restaurantes e similares,<NA>
1,01.952.454/0001-25,01.952.454 IVONE DE CASTRO,<NA>,2,20031004,19970701,<NA>,S,0,20230101,5611201,AVENIDA DOUTOR LAURO CORREA DA SILVA 2800,BAIRRO DA GRAMINHA,LIMEIRA,SP,13481636,19-34514454,Restaurantes e similares,STOAN@TERRA.COM.BR


In [22]:
MEIS["EMAIL"].fillna("0", inplace=True)

MEIS_COM_EMAIL = MEIS[~MEIS["EMAIL"].isin([0, "0"])]
MEIS_COM_EMAIL["EMAIL"].value_counts()

EMAIL
EMPRESA.HUP@GMAIL.COM                     51
ERMANIMEI2021@GMAIL.COM                   50
JCCONTABILIDADEDORENSE@HOTMAIL.COM        32
SALADOEMPREENDEDORVALENCIANO@GMAIL.COM    32
PACHECO.CONTABIL2015@GMAIL.COM            29
                                          ..
evenildozilse@gmail.com                    1
KAIQUE.BARBOSA26@HOTMAIL.COM               1
denisecristhina9@gmail.com                 1
fernaandasouusa2404@gmail.com              1
DAIANASVIDALL0@GMAIL.COM                   1
Name: count, Length: 639337, dtype: Int64

In [23]:
MEIS_COM_EMAIL_SEM_CONTABIL = MEIS_COM_EMAIL[~MEIS_COM_EMAIL["EMAIL"].str.contains("contabi")].copy()
MEIS_COM_EMAIL_SEM_CONTABIL = MEIS_COM_EMAIL_SEM_CONTABIL[~MEIS_COM_EMAIL_SEM_CONTABIL["EMAIL"].str.contains("CONTABI")]
MEIS_COM_EMAIL_SEM_CONTABIL["EMAIL"].value_counts()

EMAIL
EMPRESA.HUP@GMAIL.COM                     51
ERMANIMEI2021@GMAIL.COM                   50
SALADOEMPREENDEDORVALENCIANO@GMAIL.COM    32
OFICIALCEBRASCON@GMAIL.COM                27
SERGIO@ALTINFORMATICA.COM.BR              26
                                          ..
RAQQUELSTB2018@GMAIL.COM                   1
WASHINGTONHIROSHIHORITA@GMAIL.COM          1
JOYCEKARLAVENDAS@OUTLOOK.COM               1
ELEXSANDRE@YAHOO.COM                       1
DAIANASVIDALL0@GMAIL.COM                   1
Name: count, Length: 622509, dtype: Int64

In [24]:

MEIS_COM_EMAIL_SEM_CONTABIL.to_csv("BASE_MEI_RFB_SEM_EMAIL_CONTABIL.csv", sep=";", index=False)